In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import cufflinks as cf
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'browser'


import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")
pd.set_option('display.max_columns', None)

In [3]:
df  = pd.read_csv('Copy of amazon.csv')
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [4]:
df = df.sort_values("wilson_lower_bound", ascending=False)
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [5]:
def missing_values_analysis(df):
    na_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns].isnull().sum().sort_values(ascending=True)
    ratio = (df[na_columns].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

def check_dataframe(df, head=5, tail=5):
    print("SHAPE".center(82, '~'))
    print("Rows: {}".format(df.shape[0]))
    print("Columns: {}".format(df.shape[1]))
    print("TYPES".center(82, '~'))
    print(df.dtypes)
    print("MISSING VALUES".center(82, '~'))
    print(missing_values_analysis(df))
    print("DUPLICATED VALUES".center(83, '~'))
    print(df.duplicated().sum())
    print("QUANTILES".center(82, '~'))
    # Select only numeric columns for quantile calculation
    numeric_df = df.select_dtypes(include=np.number)
    print(numeric_df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_dataframe(df)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rows: 4915
Columns: 11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~MISSING VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Missing Values  Ratio
reviewerName               1   0.02
reviewText                 1   0.02
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATED VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~QUANTILES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       0.00  0.05   0.50        0.95       0.99         1.00
overall 

In [6]:
def check_class(dataframe):
  nunique_df = pd.DataFrame({'Variable': dataframe.columns,
                             'Classes': [dataframe[i].nunique() \
                                         for i in dataframe.columns]})
  nunique_df = nunique_df.sort_values('Classes', ascending= False)
  nunique_df = nunique_df.reset_index(drop=True)
  return nunique_df

check_class(df)


,Variable,Classes
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

colors = ["#1F77B4", "#FF7F0E", "#2CA02C", "#D62728", "#9467BD", "#8C564B"]

def categorical_variable_summary(df, column_name):
    # Create a subplot layout with two plots (Bar and Pie)
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Countplot', 'Percentage'),
        specs=[[{'type': 'xy'}, {'type': 'domain'}]]
    )

    # Bar plot for counts
    fig.add_trace(
        go.Bar(
            y=df[column_name].value_counts().values.tolist(),
            x=[str(i) for i in df[column_name].value_counts().index],
            text=df[column_name].value_counts().values.tolist(),
            textfont=dict(size=14),
            name=column_name,
            textposition='auto',
            showlegend=False,
            marker=dict(
                color=colors[:len(df[column_name].value_counts())],
                line=dict(color="#FFFFFF", width=1)
            )
        ),
        row=1, col=1
    )

    # Pie chart for percentage
    fig.add_trace(
        go.Pie(
            labels=df[column_name].value_counts().keys(),
            values=df[column_name].value_counts().values,
            textfont=dict(size=18),
            textposition='auto',
            showlegend=False,
            name=column_name,
            marker=dict(colors=colors[:len(df[column_name].value_counts())])
        ),
        row=1, col=2
    )

    # Layout settings
    fig.update_layout(
        title={
            'text': column_name,
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        template='plotly_white'
    )

    iplot(fig)




In [8]:
categorical_variable_summary(df, 'overall')

In [9]:
df.reviewText.head()

2031    [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449    I have tested dozens of SDHC and micro-SDHC ca...
4212    NOTE:  please read the last update (scroll to ...
317     If your card gets hot enough to be painful, it...
4672    Sandisk announcement of the first 128GB micro ...
Name: reviewText, dtype: object

In [10]:
review_example = df.reviewText[3022]
review_example = ''.join([char for char in review_example if char.isalpha()])
review_example


'VerygoodpriceforacardmorethannecessaryExcellentqualityworksathighspeedandmaintainingthedatafaithfully'

In [11]:
review_example = review_example.lower().split()
review_example

['verygoodpriceforacardmorethannecessaryexcellentqualityworksathighspeedandmaintainingthedatafaithfully']

In [12]:
rt = lambda x: re.sub("^[a-zA-Z]",'',str(x))
df["reviewText"] = df["reviewText"].map(rt)
df["reviewText"] = df["reviewText"].str.lower()
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ update - 6/19/2014 ]]so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,have tested dozens of sdhc and micro-sdhc car...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,ote: please read the last update (scroll to t...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"f your card gets hot enough to be painful, it ...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,andisk announcement of the first 128gb micro s...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import pandas as pd

# Assuming 'df' is your DataFrame and 'reviewText' is the column containing text data
df[['polarity', 'subjectivity']] = df['reviewText'].apply(
    lambda Text: pd.Series(TextBlob(Text).sentiment)
)

# Initialize the SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Analyze sentiment row by row
for index, row in df['reviewText'].items():  # Use .items() instead of .iteritems()
    score = analyzer.polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    
    # Determine sentiment based on scores
    if neg > pos:
        df.loc[index, 'sentiment'] = "Negative"
    elif pos > neg:
        df.loc[index, 'sentiment'] = "Positive"
    else:
        df.loc[index, 'sentiment'] = "Neutral"


In [19]:
df[df['sentiment']=="Positive"].sort_values("wilson_lower_bound",ascending=False).head(5)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
2031,"Hyoun Kim ""Faluzure""",5,[[ update - 6/19/2014 ]]so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544,0.156702,0.558574,Positive
3449,NLee the Engineer,5,have tested dozens of sdhc and micro-sdhc car...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519,0.091564,0.514756,Positive
4212,SkincareCEO,1,ote: please read the last update (scroll to t...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139,0.208749,0.501294,Positive
317,"Amazon Customer ""Kelly""",1,"f your card gets hot enough to be painful, it ...",09-02-2012,1033,422,73,495,349,0.852525,0.818577,0.151042,0.470566,Positive
4672,Twister,5,andisk announcement of the first 128gb micro s...,03-07-2014,158,45,4,49,41,0.918367,0.808109,0.164206,0.493237,Positive


In [26]:
df[df['sentiment']=="Negative"].sort_values("wilson_lower_bound",ascending=False).head(5)


,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
3807,R. Heisler,3,bought this card to replace a lost 16 gig in ...,27-02-2013,649,22,3,25,19,0.880000,0.700442,0.194643,0.675000,Negative
1072,Crysis Complex,5,hat more can i say? the 64gb micro sd works fo...,10-05-2012,942,5,0,5,5,1.000000,0.565518,0.375000,0.452381,Negative
1142,Daniel Pham(Danpham_X @ yahoo. com),5,s soon as i saw that this card was announced o...,04-02-2014,307,5,0,5,5,1.000000,0.565518,0.058929,0.592857,Negative
2695,kinematic,5,id not work when first inserted into my samsun...,09-08-2012,851,6,1,7,5,0.857143,0.486872,0.130000,0.364444,Negative
938,Chris G,5,bought the 64gb version of this card to incre...,14-10-2012,785,3,0,3,3,1.000000,0.438503,0.000000,0.000000,Negative


In [25]:
df[df['sentiment']=="Neutral"].sort_values("wilson_lower_bound",ascending=False).head(5)


,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
1242,"David S. Winokur ""Photographer, Luthier""",5,ot this for my phone. at first it wouldn't wor...,22-11-2012,746,2,0,2,2,1.000000,0.342380,0.346591,0.509470,Neutral
4747,Vlado,1,"32 gb card is unredable...tablet cant read it,...",19-09-2012,810,3,2,5,1,0.600000,0.230724,-0.333333,0.283333,Neutral
125,Alexander Stevens,5,an,21-08-2012,839,2,1,3,1,0.666667,0.207660,0.000000,0.000000,Neutral
1585,Eric,5,hat can you say about a memory card. i took it...,06-12-2013,367,1,0,1,1,1.000000,0.206549,0.000000,0.000000,Neutral
1511,"Edward C ""TekDragon""",5,ought two of these. one for my galaxy note 3 a...,12-07-2013,514,1,0,1,1,1.000000,0.206549,0.095238,0.330952,Neutral


In [20]:
categorical_variable_summary(df,'sentiment')